In [2]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from googlesearch import search
import random
from concurrent.futures import ThreadPoolExecutor
from requests.exceptions import RequestException

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
}

# Reading proxies
file_path = 'proxy_10sec.txt'
with open(file_path, 'r') as file:
    proxy_addresses = [line.strip() for line in file]
proxy_list = proxy_addresses

date_pattern = r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{4}\b'
rating_pattern = r'bubble_(\d{2})'

selected_proxy = random.choice(proxy_list)

failed_urls = []

def scrape_reviews(url, base_url):
    scraped_data = []

    try:
        r = requests.get(url, headers=headers, proxies={'http': selected_proxy, 'http': selected_proxy}, timeout=20)
        r.raise_for_status()  # Raise an exception for HTTP errors

        s = BeautifulSoup(r.content, 'html.parser')
        reviews_card = s.find_all('div', class_='YibKl MC Gi z Z BB pBbQr lqYst')
        for i in reviews_card:
            review_data = {}
            review_data['TRIP_ADVISOR_HREF'] = base_url
            date = i.find('div', class_='cRVSd').text
            date_match = re.search(date_pattern, date)
            if date_match:
                review_data['REVIEW_DATE'] = date_match.group(0)
            else:
                review_data['REVIEW_DATE'] = "No date found."

            rating = i.find('div', class_='Hlmiy F1').find('span')
            match = re.search(rating_pattern, str(rating))
            if match:
                review_data['RATING'] = int(match.group(1)) // 10
            else:
                review_data['RATING'] = "Rating value not found."

            review_data['REVIEW_TITLE'] = i.find('div', class_='KgQgP MC _S b S6 H5 _a').text
            review_data['REVIEW_TEXT'] = i.find('span', class_='QewHA H4 _a').text
            scraped_data.append(review_data)
    except RequestException as e:
        failed_urls.append(url)
    return scraped_data


FileNotFoundError: [Errno 2] No such file or directory: 'proxy_10sec.txt'

In [ ]:
import pandas as pd
all_reviews_data = []
df = pd.read_csv('Ratings.csv')
df = df[5500:]
print(len(df))
reviews_list = df['TOTAL_REVIEWS'].to_list()
href_list= df['WEBSITE'].to_list()


for a, b in zip(reviews_list, href_list):
    # this checks if is a number
    if isinstance(a, str) and (a.isalpha() or a.isdigit()):
        if int(a) < 30:
            # checks if review is less than 30
            reviews_index = b.find('-Reviews-')
            if reviews_index != -1:
                base_url = b[:reviews_index + 9]  # Include '-Reviews-' and the following character
                review_page_number = b[reviews_index + 9:-5]  # Exclude '.html' at the end
                for j in range(0, int(a), 10):
                    modified_url = f"{base_url}or{j}-{review_page_number}.html"
                    # Call the scrape_reviews function and append its result to the all_reviews_data list
                    all_reviews_data.extend(scrape_reviews(modified_url, b))
        else:
            reviews_index = b.find('-Reviews-')
            if reviews_index != -1:
                base_url = b[:reviews_index + 9]  # Include '-Reviews-' and the following character
                review_page_number = b[reviews_index + 9:-5]  # Exclude '.html' at the end
                # Generate URLs with different offsets using f-strings and range function
                for j in range(0, 30, 10):
                    modified_url = f"{base_url}or{j}-{review_page_number}.html"
                    # Call the scrape_reviews function and append its result to the all_reviews_data list
                    all_reviews_data.extend(scrape_reviews(modified_url, b))

        while failed_urls:
            url = failed_urls.pop(0)
            all_reviews_data.extend(scrape_reviews(url, b))
    else:
        pass


In [ ]:
import pandas as pd
df2 = pd.DataFrame(all_reviews_data)
df2.to_csv('last.csv', index = False)

In [ ]:
unique_values = pd.unique(df2['TRIP_ADVISOR_HREF'])
# Print unique values
print(len(unique_values))

2


In [ ]:
df = pd.read_csv('Ratings.csv')
df = df[5500:]
reviews_list = df['TOTAL_REVIEWS'].to_list()
href_list= df['WEBSITE'].to_list()
t_list = []
for a, b in zip(reviews_list, href_list):
    # this checks if is a number
    if isinstance(a, str) and (a.isalpha() or a.isdigit()):
      if int(a) >0:
        t_list.append(b)
c = [x for x in t_list if x not in unique_values]

print(c)

['https://www.tripadvisor.com/Hotel_Review-g55822-d613650-reviews-budget_inn_fairfield-fairfield_texas.html']


In [ ]:
print(len(set(t_list)))
print(len(unique_values))

286
285


In [ ]:
import csv
import pandas as pd

# List of CSV file names
csv_files = [
    'R_0-10.csv',
    'R_10-100.csv',
    'R_100-200.csv',
    'R_200-500.csv',
    'R_500-1000.csv',
    'R_1000-1500.csv',
    'R_1500-2000.csv',
    'R_2000-2500.csv',
    'R_2500-2510.csv',
    'R_2510-2530_real.csv',
    'R_2530-2600.csv',
    'R_2600-3000.csv',
    'R_3000-3500.csv',
    'R_3500-4000.csv',
    'R_4000-4500.csv',
    'R_4500-5000.csv',
    'R_5000-5500.csv',
    'R_5500-last.csv',
    'last.csv'
]

# Read all CSV files into DataFrames
dfs = []
for file_name in csv_files:
    with open(file_name, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        header = next(csv_reader)  # Read the header
        data = [row for row in csv_reader]  # Read the remaining rows
        df = pd.DataFrame(data, columns=header)
        dfs.append(df)
    print(file_name, 'done')

# Concatenate all DataFrames vertically
result = pd.concat(dfs, ignore_index=True)
print(len(result))

result.to_csv('Reviews.csv', index=False)


R_0-10.csv done
R_10-100.csv done
R_100-200.csv done
R_200-500.csv done
R_500-1000.csv done
R_1000-1500.csv done
R_1500-2000.csv done
R_2000-2500.csv done
R_2500-2510.csv done
R_2510-2530_real.csv done
R_2530-2600.csv done
R_2600-3000.csv done
R_3000-3500.csv done
R_3500-4000.csv done
R_4000-4500.csv done
R_4500-5000.csv done
R_5000-5500.csv done
R_5500-last.csv done
last.csv done
125694


In [ ]:
import pandas as pd

# Read the first CSV file
df1 = pd.read_csv('Ratings.csv')

# Read the second CSV file
df2 = result

# Create an Excel writer object
with pd.ExcelWriter('RevandRatings_.xlsx') as writer:
    # Write the first DataFrame to the Excel file as Sheet1
    df1.to_excel(writer, sheet_name='Sheet1', index=False)

    # Write the second DataFrame to the Excel file as Sheet2
    df2.to_excel(writer, sheet_name='Sheet2', index=False)

print("Excel file created successfully.")


Excel file created successfully.
